# Step 1: generate contact maps using well-trained model 


## 1. Load packages

In [ ]:
##########################
#    Loading packages    #
##########################

# 1. Load packages

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import randn
from torch.nn import MSELoss
import torch.optim as optim
from torch.optim import Adam
from torch.utils.data import DataLoader
import numpy as np
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
import random
import pandas as pd
import seaborn as sns
import time
import pickle
from datetime import datetime
!pip install hickle
import hickle as hkl
from torch.autograd import Variable
import gzip
import sys
import os 
from sklearn.decomposition import TruncatedSVD, PCA
torch.set_default_tensor_type(torch.DoubleTensor)
!pip install pyBigWig
import pyBigWig

# 2. Load data - part 2

!wget https://s3.amazonaws.com/hicfiles.tc4ga.com/public/juicer/juicer_tools_1.22.01.jar
!wget http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/genes/hg38.refGene.gtf.gz
!gunzip /content/hg38.refGene.gtf.gz

chrom_list = ["chr"+str(i) for i in range(1,23)] #for human hg38
length_list = [248956422,242193529,198295559,190214555,181538259,170805979,159345973,145138636,
               138394717,133797422,135086622,133275309,114364328,107043718,101991189,90338345,
               83257441,80373285,58617616,64444167,46709983,50818468]
chrom_len_dict = dict(zip(chrom_list,length_list))

--2021-12-16 20:53:31--  https://s3.amazonaws.com/hicfiles.tc4ga.com/public/juicer/juicer_tools_1.22.01.jar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.186.141
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.186.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36188666 (35M) [application/x-www-form-urlencoded]
Saving to: ‘juicer_tools_1.22.01.jar’

juicer_tools_1.22.0 100%[===================>]  34.51M  30.3MB/s    in 1.1s    

2021-12-16 20:53:32 (30.3 MB/s) - ‘juicer_tools_1.22.01.jar’ saved [36188666/36188666]

--2021-12-16 20:53:32--  http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/genes/hg38.refGene.gtf.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23614606 (23M) [application/x-gzip]
Saving to: ‘hg38.refGene.gtf.gz’

hg38.refGene.gtf.gz 10

## 2. Load trained model 

In [ ]:
cd /content/drive/MyDrive/Research/Predict2D/notebooks5/util_functions

/content/drive/MyDrive/Research/Predict2D/notebooks5/util_functions


In [ ]:
###############################
#    Loading trained model    #
###############################

#1. import model architecture and util functions 

from model_architecture_util import *

#2. load model 

wsize = 14000
net = Net(window_size=wsize)
restore(net,'/content/trained_model')
net.eval()

Restoring:
conv1.conv.weight -> 	torch.Size([70, 5, 17]) = 0MB
conv1.conv.bias -> 	torch.Size([70]) = 0MB
conv2.conv.weight -> 	torch.Size([90, 70, 7]) = 0MB
conv2.conv.bias -> 	torch.Size([90]) = 0MB
conv3.conv.weight -> 	torch.Size([70, 90, 5]) = 0MB
conv3.conv.bias -> 	torch.Size([70]) = 0MB
conv4.conv.weight -> 	torch.Size([20, 70, 5]) = 0MB
conv4.conv.bias -> 	torch.Size([20]) = 0MB
rnn1.weight_ih_l0 -> 	torch.Size([4800, 900]) = 17MB
rnn1.weight_hh_l0 -> 	torch.Size([4800, 1200]) = 23MB
rnn1.bias_ih_l0 -> 	torch.Size([4800]) = 0MB
rnn1.bias_hh_l0 -> 	torch.Size([4800]) = 0MB
rnn1.weight_ih_l0_reverse -> 	torch.Size([4800, 900]) = 17MB
rnn1.weight_hh_l0_reverse -> 	torch.Size([4800, 1200]) = 23MB
rnn1.bias_ih_l0_reverse -> 	torch.Size([4800]) = 0MB
rnn1.bias_hh_l0_reverse -> 	torch.Size([4800]) = 0MB
rnn2.weight_ih_l0 -> 	torch.Size([4800, 2400]) = 46MB
rnn2.weight_hh_l0 -> 	torch.Size([4800, 1200]) = 23MB
rnn2.bias_ih_l0 -> 	torch.Size([4800]) = 0MB
rnn2.bias_hh_l0 -> 	torch.Size

Net(
  (conv1): ConvBlock(
    (conv): Conv1d(5, 70, kernel_size=(17,), stride=(1,))
    (act): ReLU()
    (pool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (do1): Dropout(p=0.1, inplace=False)
  (conv2): ConvBlock(
    (conv): Conv1d(70, 90, kernel_size=(7,), stride=(1,))
    (act): ReLU()
    (pool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (do2): Dropout(p=0.1, inplace=False)
  (conv3): ConvBlock(
    (conv): Conv1d(90, 70, kernel_size=(5,), stride=(1,))
    (act): ReLU()
    (pool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (do3): Dropout(p=0.1, inplace=False)
  (conv4): ConvBlock(
    (conv): Conv1d(70, 20, kernel_size=(5,), stride=(1,))
    (act): ReLU()
  )
  (pool): AdaptiveMaxPool1d(output_size=45)
  (do4): Dropout(p=0.1, inplace=False)
  (rnn1): LSTM(900, 1200, batch_first=True, bidirectional=True)
  (rnn2): LSTM(2400, 1200, batch_first=True, bidirectional=Tru

## 3. Generating predictions using trained model 

Please 

In [ ]:
####################################################
#    Generating predictions using trained model    #
####################################################

#1. import util functions

from generate_predictions_util import *

#2. generate predictions for chromosomes

pth = '/content/Ground_truth_HiC_diagonal_list.txt'
for i in [1,23]:
  chrom = "chr" + str(i)
  print(chrom,datetime.now())
  results_generation(chrom = chrom,
                     cell_type = "GM12878", 
                     bwfile_dir = "/content/bigWig/GRCh38",
                     submatrix_location = "/content/loc1.txt", 
                     assemble_matrix_location = "/content/loc2.txt",
                     ground_truth_file = pth, 
                     ground_truth_location = "/content/loc3.txt", 
                     window_size = wsize) #normcounts, zvalue, zfull